# eRADAR 

In [0]:
import numpy as np
import pandas as pd

Load the patient data.
- /Volumes/biomedicalinformatics_analytics/dev_lab_johnson/watch/pair_watch_final_2025.csv

In [0]:
pt_data = pd.read_csv("/Volumes/biomedicalinformatics_analytics/dev_lab_johnson/watch/pair_watch_final_2025.csv", header=0, index_col=0)
pt_data.head()

Apply the eRADAR model to the patient data.

In [0]:
class eRADAR:
    def __init__(self):
        self.intercept = -11.83
        self.weights = [0.11, -0.10, 0.28, 0.18, 0.06, 0.34, -0.11, 0.05, -0.15, 0.01, -0.17, -0.05, -0.34, -0.07, -0.25, 0.46, 0.03, 0.20, -0.15, 0.03, 0.24, 0.29, -0.11, -0.09, -0.55, 0.34, 0.00, -0.13, 0.47, 0.58, 0.03]

    def score(self, x):
        return 1 / (1 + np.exp(-self.intercept - np.dot(self.weights, x)))

In [0]:
model = eRADAR()

pt_scores = pt_data[["age", "female", "dx_chf", "dx_cebv", "dx_diab", "dx_diab_c", "dx_cpd", "dx_hypothyoidism", "dx_rf", "dx_lymphoma", "dx_solidtumor", "dx_ra", "dx_weightloss", "dx_fluid", "dx_anemia", "dx_bipolar", "dx_depression", "dx_trauma", "dx_tobacco", "dx_atrial", "dx_gait", "low_bmi", "high_bmi", "hbp", "op_primary_care_visit", "ed_visit", "home_visit", "pt_visit", "cognitive_visit", "med_antidepressant", "med_sleepaid"]].apply(model.score, axis=1)

Analysis.

In [0]:
import matplotlib.pyplot as plt

In [0]:
qs = [99] + list(range(95, 45, -5))
thresholds = np.percentile(pt_scores, qs)
n_patients = [(pt_scores >= t).sum() for t in thresholds]
ages = [(round(pt_data.loc[(pt_scores >= t), "age"].mean()), round(pt_data.loc[(pt_scores >= t), "age"].std(), 2)) for t in thresholds]
females = [round(pt_data.loc[(pt_scores >= t), "female"].sum() / pt_data.loc[(pt_scores >= t)].shape[0], 2) for t in thresholds]

pd.DataFrame({"Cutoff Percentile": qs, "Cutoff": thresholds, "Num. Patients": n_patients, "Age (mean, std)": ages, "Pct. Female": females})

In [0]:
plt.figure(figsize=(15, 5))
plt.hist(pt_scores, bins=500)

for i in [0, 1, 2, 4, 9]:
    plt.axvline(thresholds[i], color="red", linestyle="--", alpha=0.6)
    plt.text(thresholds[i], 1.02, f"{qs[i]}th", transform=plt.gca().get_xaxis_transform(), color="red", rotation=90, ha="center")

plt.xlim([0, 1])
plt.xticks(np.arange(0.0, 1.1, 0.1))
plt.yscale("log")
plt.xlabel("eRADAR Score")
plt.ylabel("Frequency (Num. Patients)")
plt.title("Distribution of eRADAR Scores for Penn Patients", y=1.05)
plt.grid()
plt.show()